In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [ ]:
url = "https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union"
page = urlopen(url)
soup = BeautifulSoup(page, "lxml")
urls = [i.get("href") for i in soup.select("table a")]
urls = [u for u in urls if u is not None and u.startswith("http")]

In [ ]:
l = []
for url in urls:
    print(url)
    page = urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    transcript = " ".join([e.get_text(strip=True) for e in soup.select("div.field-docs-content p")])
    title = soup.select_one("div.field-ds-doc-title h1").string
    date = soup.select_one("div.field-docs-start-date-time span").string # todo: convert to date format
    name = soup.select_one("div.field-title a").string
    l.append({
        "url": url,
        "title": title,
        "date": date,
        "name": name,
        "transcript": transcript
    })

In [ ]:
import json
with open('transcripts-fixed.json', 'w') as f:
    json.dump(l, f, indent=2)